<a href="https://colab.research.google.com/github/SojeongShin/NLP_2023/blob/main/14_1_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [자연어처리]
# 14주차(14-1). GPT

# 202001581 신소정 <- 본인의 학번, 이름으로 변경하여 제출
* **모든 셀 실행** 후 제출하시기 바랍니다.
* **실습 (14-1)**이 있습니다. (제출 기한: 12/5(화) 23시 59분까지)

# 이제 Colab 사용에 모든 수강생분들이 어느정도 익숙해 졌을 거라 생각됩니다!
> ##### 중간 중간 **'### ... 이 부분을 완성하시오'** 라는 부분의 코드를 완성해야 합니다.




참고 자료 : https://huggingface.co/blog/how-to-generate

사전 학습된 한국어 GPT-2를 이용하여 다음 문장 예측을 실습해봅시다. 이번 실습을 위해서만이 아니라 앞으로 사전 학습된 GPT를 사용할 때는 transformers라는 패키지를 자주 사용하게 됩니다. 실습 환경에 transformers 패키지를 설치해둡시다.

In [2]:
! pip install transformers ### ... 이 부분을 완성하시오

# [런타임] - [런타임 유형 변경] - [T4 GPU]

# 1. KoGPT2로 문장 생성하기

transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT와 마찬가지로 GPT는 이미 누군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 합니다.

In [3]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

TFGPT2LMHeadModel.from_pretrained('GPT 모델 이름')을 넣으면 두 개의 문장이 이어지는 문장 관계인지 여부를 판단하는 GPT 구조를 로드합니다. AutoTokenizer.from_pretrained('모델 이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드합니다.

In [4]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2') ### ... 이 부분을 완성하시오

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [5]:
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True) ### ... 이 부분을 완성하시오

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

일반적으로 PyTorch 모델을 TensorFlow 모델로 변환하는 과정에서 흔히 발생하는 워닝 메시지. 변환 과정에서 모든 가중치가 완벽하게 일치하지 않을 수 있다는 것을 의미. 대부분의 경우 모델은 제대로 작동.

GPT가 생성할 문장의 방향성을 알려주기 위해서 시작 문자열을 정해줍니다. 여기서는 '건강한 몸을 만들기 위해서는' 이라는 문자열을 주고 GPT에게 이어서 문장을 생성해보라고 해봅시다.

In [7]:
sent = '건강한 몸을 만들기 위해서는' ### ... 이 부분을 완성하시오

In [8]:
input_ids = tokenizer.tokenize(sent) ### ... 이 부분을 완성하시오
print(input_ids) ### ... 이 부분을 완성하시오

['▁건강한', '▁몸을', '▁만들기', '▁위해서는']


GPT의 입력으로는 정수 인코딩 된 결과가 입력되어야 하므로 tokenizer.encode()를 통해서 '건강한 몸을 만들기 위해서는'이라는 문자열을 정수 시퀀스로 변환해줍니다.

In [9]:
input_ids = tokenizer.encode(sent) ### ... 이 부분을 완성하시오
input_ids = tf.convert_to_tensor([input_ids])
print(input_ids) ### ... 이 부분을 완성하시오

tf.Tensor([[22159 14226 15381 11357]], shape=(1, 4), dtype=int32)


22159 14226 15381 11357라는 4개의 정수 시퀀스를 얻습니다.

해당 정수 시퀀스를 GPT의 입력으로 사용하여 GPT가 이어서 문장을 생성하도록 해봅시다. 주어진 문장으로부터 이어서 문장을 생성하도록 하는 것은 model.generate()를 사용합니다.

In [11]:
output = model.generate(input_ids,
                   max_length=128,
                   repetition_penalty=2.0) ### ... 이 부분을 완성하시오
output_ids = output.numpy().tolist()[0]
print(output_ids) ### ... 이 부분을 완성하시오

[22159, 14226, 15381, 11357, 23879, 39306, 25235, 9376, 33820, 10362, 9430, 15967, 38594, 12443, 8137, 17764, 9033, 9199, 15177, 26421, 387, 15386, 409, 7162, 9346, 7071, 11913, 9091, 9377, 12486, 10108, 24873, 7198, 9391, 37611, 9676, 14458, 9492, 20305, 10090, 10254, 9178, 7335, 8704, 41845, 18418, 9277, 18324, 8643, 7122, 8135, 9351, 16709, 10335, 18380, 16286, 13940, 19671, 9048, 18281, 8648, 44869, 6824, 12923, 13600, 9600, 39137, 8066, 23865, 8470, 10461, 9164, 7609, 8397, 9225, 9029, 45837, 9052, 48276, 15528, 10386, 10988, 8711, 22452, 8765, 27929, 12972, 12364, 23511, 8450, 16803, 21302, 6903, 9410, 13041, 25606, 49019, 7586, 9752, 8143, 10850, 10007, 26006, 27147, 8146, 26523, 9359, 9341, 13219, 35943, 14441, 8000, 13532, 13565, 11634, 12268, 10279, 36122, 9960, 8405, 11545, 13235, 7849, 17304, 14355, 10826, 43075, 7235]


기존의 22159 14226 15381 11357 라는 4개의 정수 시퀀스 뒤에도 여러 정수들이 추가로 생성된 것을 볼 수 있습니다. 정수들이 단순히 나열된 것만으로는 GPT가 실제로 어떤 문장을 생성했는지 알기 어려우니 해당 정수 시퀀스를 한국어로 변환해봅시다. 이 과정은 tokenizer.decode()를 사용하여 가능합니다.

In [12]:
print(tokenizer.convert_ids_to_tokens(output_ids)) ### ... 이 부분을 완성하시오

['▁건강한', '▁몸을', '▁만들기', '▁위해서는', '▁무엇보다', '▁규칙적인', '▁운동과', '▁식', '이요', '법을', '▁통해', '▁균형', '▁잡힌', '▁영양', '을', '▁섭취', '하는', '▁것이', '▁중요하다.\n', '특히', ',', '▁비타민', 'C', '는', '▁체', '내', '▁에너지', '▁생', '성을', '▁촉진', '시켜', '▁신진', '대', '사를', '▁원활하게', '▁하고', '▁면역', '력을', '▁높여', '주는', '▁역할을', '▁한다.\n', '또', '한', '▁항산화', '▁작용을', '▁하는', '▁폴리', '페', '놀', '은', '▁우리', '▁몸의', '▁세포', '막을', '▁구성하는', '▁필수', '▁성분', '이다.\n', '폴리', '펩', '타이드', '가', '▁풍부한', '▁식품', '으로는', '▁토마토', '와', '▁브로', '콜', '리,', '▁양', '배', '추', '▁등이', '▁있다.\n', '토마토', '에는', '▁식이', '섬유', '소가', '▁풍부', '해', '▁피로', '회', '복에', '▁도움을', '▁준다.\n', '브로', '컬', '리에는', '▁칼슘', '과', '▁철', '분이', '▁풍부하다.\n', '비타', '민', '▁C', '의', '▁흡', '수를', '▁돕는', '▁칼륨', '이', '▁풍부하게', '▁들어', '▁있어', '▁뼈', '▁건강에', '▁좋다.\n', '양', '파는', '▁섬유', '질이', '▁많아', '▁혈', '압을', '▁낮', '춰', '주고', '▁혈액', '순', '환을', '▁도와', '준다.\n', '마늘', '도']


In [13]:
tokenizer.decode(output_ids) ### ... 이 부분을 완성하시오

'건강한 몸을 만들기 위해서는 무엇보다 규칙적인 운동과 식이요법을 통해 균형 잡힌 영양을 섭취하는 것이 중요하다.\n특히, 비타민C는 체내 에너지 생성을 촉진시켜 신진대사를 원활하게 하고 면역력을 높여주는 역할을 한다.\n또한 항산화 작용을 하는 폴리페놀은 우리 몸의 세포막을 구성하는 필수 성분이다.\n폴리펩타이드가 풍부한 식품으로는 토마토와 브로콜리, 양배추 등이 있다.\n토마토에는 식이섬유소가 풍부해 피로회복에 도움을 준다.\n브로컬리에는 칼슘과 철분이 풍부하다.\n비타민 C의 흡수를 돕는 칼륨이 풍부하게 들어 있어 뼈 건강에 좋다.\n양파는 섬유질이 많아 혈압을 낮춰주고 혈액순환을 도와준다.\n마늘도'

'건강한 몸을 만들기 위해서는' 이라는 문자열에 이어서 그 뒤에 건강한 몸을 만들기 위한이라는 문맥에 맞는듯한 글들을 생성합니다.

물론 GPT가 생성한 문장들은 문맥상 그럴듯해 보이지만 실제 사실 여부와는 다를 수 있으니 이 점은 늘 주의해야 합니다.

# 2. Numpy로 Top 5 뽑기

GPT는 기본적으로 이전 단어들로부터 다음 단어를 예측하는 언어 모델(Language Model)입니다. 위의 실습에서 확인한 바와 같이 '건강한 몸을 만들기 위해서는' 이라는 입력을 넣었을 때 GPT는 다음 단어로 '무엇보다' 라는 단어를 예측했었는데요.


실제로는 수많은 후보의 다음 단어들이 있었지만, 그 중 가장 확률이 높은 단어. 즉, Top 1의 단어인 '무엇보다'를 예측한 것입니다. 그렇다면 다음 단어로 또 어떤 후보들이 있었는지 Top 5의 단어들을 뽑아봅시다.


model()에다가 '건강한 몸을 만들기 위해서는'의 정수 인코딩 된 결과를 입력으로 넣은 후 가장 확률이 높은 Top 5의 단어들을 뽑아냅니다.

In [14]:
import numpy as np

In [15]:
output = model(input_ids)

In [20]:
output.logits ### ... 이 부분을 완성하시오

<tf.Tensor: shape=(1, 4, 51200), dtype=float32, numpy=
array([[[-6.1596045, -5.6269374, -5.299072 , ...,  0.1811116,
         -4.9666777, -3.0936294],
        [-4.6646895, -5.924712 , -4.8221292, ...,  2.4812758,
         -3.5151467,  2.3900423],
        [-5.0853715, -4.0964794, -6.0712833, ...,  1.1920158,
         -2.90183  , -3.3687851],
        [-6.2043076, -6.284622 , -6.3270826, ...,  1.8252198,
         -3.9164758, -2.7165358]]], dtype=float32)>

In [22]:
output.logits.shape ### ... 이 부분을 완성하시오 51200개

TensorShape([1, 4, 51200])

In [23]:
top5 = tf.math.top_k(output.logits[0, -1], k=5)
top5

TopKV2(values=<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([10.895334 , 10.395889 ,  9.945646 ,  9.840968 ,  9.8311825],
      dtype=float32)>, indices=<tf.Tensor: shape=(5,), dtype=int32, numpy=array([23879, 22159, 20030, 11342, 39306], dtype=int32)>)

그 후 Top 5의 단어를 한국어로 변환하여 출력해봅시다.

In [24]:
tokenizer.convert_ids_to_tokens(top5.indices.numpy()) ### ... 이 부분을 완성하시오

['▁무엇보다', '▁건강한', '▁무엇보다도', '▁건강', '▁규칙적인']

'무엇보다' 라는 단어 외에도 '건강한', '무엇보다도', '건강', '규칙적인' 이라는 4개의 단어가 높은 확률로 선택될 수 있었음을 보여줍니다.

# 3. Numpy Top 5 중 랜덤 단어로 문장 생성해보기

앞서 문장을 생성했을 당시에는 각 시점(time step)마다 가장 확률이 높은 단어를 예측했지만, 이번에는 매 시점마다 Top 5개의 단어들 중에서 랜덤으로 선택하는 방식을 택하여 문장을 생성해봅시다.

In [25]:
sent = '건강한 몸을 만들기 위해서는' ### ... 이 부분을 완성하시오
input_ids = tokenizer.encode(sent) ### ... 이 부분을 완성하시오

In [26]:
import random

while len(input_ids) < 50:
    output = model(np.array([input_ids]))
    top5 = tf.math.top_k(output.logits[0, -1], k=5)
    token_id = random.choice(top5.indices.numpy())
    input_ids.append(token_id)

In [27]:
tokenizer.decode(input_ids) ### ... 이 부분을 완성하시오

'건강한 몸을 만들기 위해서는 건강하고 아름다운 몸을 가꾸기 위해서는 올바른 생활 습관, 바른 생활 습관, 건강 습관 등을 통해 건강을 지키면서 올바른 생활을 해야 한다고 전문의는 말했다.\n이어 그는 "최근의 건강기능식품 시장은 건강과 관련된 다양한 제품들을 통해 다양한 제품군을 통해'

# 4. pipeline을 이용해 문장 생성해보기

In [29]:
# Use a pipeline as a high-level helper
from transformers import pipeline ### ... 이 부분을 완성하시오
pipe = pipeline("text-generation", model="skt/kogpt2-base-v2", device=0) ### ... 이 부분을 완성하시오

In [30]:
sent = "건강한 몸을 만들기 위해서는"  ### ... 이 부분을 완성하시오
generated_text = pipe(sent)

In [31]:
sent = "건강한 몸을 만들기 위해서는"
generated_text = pipe(sent, max_new_tokens=256)  ### ... 이 부분을 완성하시오
generated_text

[{'generated_text': '건강한 몸을 만들기 위해서는 단백질과 비타민이 풍부한 음식을 섭취하는 것부터 해결해야 한다.\n최근에 식생활을 통해 탄수화물과 단백질, 지방, 지방 같은 영양소 섭취가 급증하면서 칼로리가 급격히 떨어진 음식을 섭취하고 있다.\n그럼에도 칼로리가 낮은 음식의 섭취량이 늘고 있다.\n단순한 체중감량 뿐 아니라 체중감량뿐만 아니라 체중관리를 통해 식생활을 개선하는 것도 다이어트의 중요한 요소로 꼽힌다.\n칼로리 감량에 성공하기 위해서는 식이조절로 체중을 줄이되, 하루 평균 열량은 늘려야 한다.\n특히 아침식사와 저녁식사 30분 전에 반드시 균형 잡힌 식사를 해야 한다.\n채식을 통해 섭취하는 단백질의 양은 대개 하루에 최소 16g이 적당하다.\n비만을 예방하려면 하루에 열량을 10g 이상 섭취하도록 해야 한다.\n무엇보다 매일 규칙적으로 식사를 하는 것이 중요하다.\n저녁식사와 저녁식사를 할 경우 탄수화물이 부족하면 균형 있는 식사를 한다.\n단 음식을 많이 먹으면 몸에 독소가 쌓이기도 한다.\n따라서 하루 평균 열량은 16g으로 줄이는 대신 식사 시간은 30분 이상 늘려야 한다.\n영양소를 섭취한 후에 단백질과 비타민, 지방 중 어느 것이 얼마나 좋은지 꼭 확인해야 한다.\n영양소 섭취의 가장 중요한 원칙은 식습관이 몸에 필요한지를 알아보는 것이다.\n식이습관을 통해 영양소의 영양소와 에너지 균형을 찾아낼 수 있기 때문이다.\n다이어트를 결심했다면 식사량을 줄이는 것은'}]

In [32]:
print(generated_text[0]['generated_text'])

건강한 몸을 만들기 위해서는 단백질과 비타민이 풍부한 음식을 섭취하는 것부터 해결해야 한다.
최근에 식생활을 통해 탄수화물과 단백질, 지방, 지방 같은 영양소 섭취가 급증하면서 칼로리가 급격히 떨어진 음식을 섭취하고 있다.
그럼에도 칼로리가 낮은 음식의 섭취량이 늘고 있다.
단순한 체중감량 뿐 아니라 체중감량뿐만 아니라 체중관리를 통해 식생활을 개선하는 것도 다이어트의 중요한 요소로 꼽힌다.
칼로리 감량에 성공하기 위해서는 식이조절로 체중을 줄이되, 하루 평균 열량은 늘려야 한다.
특히 아침식사와 저녁식사 30분 전에 반드시 균형 잡힌 식사를 해야 한다.
채식을 통해 섭취하는 단백질의 양은 대개 하루에 최소 16g이 적당하다.
비만을 예방하려면 하루에 열량을 10g 이상 섭취하도록 해야 한다.
무엇보다 매일 규칙적으로 식사를 하는 것이 중요하다.
저녁식사와 저녁식사를 할 경우 탄수화물이 부족하면 균형 있는 식사를 한다.
단 음식을 많이 먹으면 몸에 독소가 쌓이기도 한다.
따라서 하루 평균 열량은 16g으로 줄이는 대신 식사 시간은 30분 이상 늘려야 한다.
영양소를 섭취한 후에 단백질과 비타민, 지방 중 어느 것이 얼마나 좋은지 꼭 확인해야 한다.
영양소 섭취의 가장 중요한 원칙은 식습관이 몸에 필요한지를 알아보는 것이다.
식이습관을 통해 영양소의 영양소와 에너지 균형을 찾아낼 수 있기 때문이다.
다이어트를 결심했다면 식사량을 줄이는 것은


In [33]:
sent = '건강한 몸을 만들기 위해서는 뭘 해야해?'  ### ... 이 부분을 완성하시오
generated_text = pipe(sent, max_new_tokens=256)
print(generated_text[0]['generated_text'])

건강한 몸을 만들기 위해서는 뭘 해야해?"
"뭐, 뭘 해야 해. 뭘 어떻게 해야해?"
"당신은 내가 원하든 원하지 않든 항상 내게 질문을 던지고 있어. 내가 하는 일에 내가 어떤 사람이 되려고 하는지 스스로 알 수 있어야 해."
"그런데 이젠 내가 진짜 원하는 건 뭐야?"
"난 당신 옆에 서서 당신을 좀 더 즐겁게 해보자라고 얘기해봤지. 그래서 더 잘할 수 있도록 도와달라고. 난 당신의 인생을 바꿔줄 사람을 원해."
"그래, 난 당신이 원하는 것도 알고 있지. 당신이 나를 더 잘 살게 도울지도 몰라."
"그럼, 내 진짜 목적은!"
"그럼 당신은 당신이 원하는 대로 해."
"그래, 나 좀 도와줘."
"난 당신이 원하는 대로 해요!"
"난 앞으로 당신의 일을 잘 해낼 거야!"
"그러니까 당신이 원하는 게 뭐야?"
난 그제야 이렇게 말했다.
"이제 당신이 원하는 게 뭐야?"
"당신은 내가 원하는 일을 해낼 것이고, 난 당신이 원하는 일을 할 거야!"
"그래, 난 당신의 일이 당신이 원하는 것이라 믿고 있어요. 하지만 난 나를 위해 일할 거야!"
내 목표는 정말 내가 원하는 일을 하는 거지.
난 내일을 위해


In [34]:
sent = "안녕하세요"
generated_text = pipe(sent, max_new_tokens=256)
print(generated_text[0]['generated_text'])

안녕하세요!
오늘은 더위먹으면서,
살짝 더운 날씨가 계속되자!
미리 준비해둔 텀블러에...
#마음밥으로 #가까운 #맛집 #맛스타그램 #인스타친구 #맛집탐방 #gate #일상 #데일리 #소통stagram #먹스타그램 #푸드스타그램 #맛스타그램 #술스타그램 #푸드파이터 #instfood #foodie #foodporn #foodstagram #소통 #좋아요
#팔로우 #맞팔 #선팔 #instafood #delicious #instadaily #셀스타그램 #맞팔환영 #선팔하면맞팔 내가 살찌고싶다
일단 #슈퍼마리오브라더스
#스콜피오 #스콜피오 #슈퍼 마리오브라더스
#스콜피오브라더스마리오브랜드 #슈퍼마리오브라더스마리오브랜드 #브라더마리오브라


In [35]:
# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-generation", model="heegyu/polyglot-ko-1.3b-chat", device=0) ### ... 이 부분을 완성하시오

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [36]:
sent = "건강한 몸을 만들기 위해서는"
generated_text = pipe(sent, max_new_tokens=256)
print(generated_text[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


건강한 몸을 만들기 위해서는 적절한 운동과 영양 섭취가 필요합니다.

2. 스트레스 관리: 스트레스는 건강에 부정적인 영향을 미치는 요인 중 하나입니다. 스트레스를 관리하기 위해서는 명상, 운동, 취미활동 등을 통해 스트레스를 해소하는 것이 중요합니다.

3. 수면: 수면 부족은 건강에 부정적인 영향을 미칩니다. 충분한 수면을 취하지 않으면 면역체계가 약화되고, 심혈관 질환의 위험이 증가할 수 있습니다.

4. 영양 섭취: 건강한 식단은 건강한 체중을 유지하는 데 도움이 됩니다. 균형 잡힌 식단은 영양소를 충분히 공급하고, 혈당 조절을 도와줍니다.

5. 운동: 운동은 건강한 체중을 유지하는 데 도움이 됩니다. 유산소 운동과 근력 운동을 병행하면 근육량을 증가시키고, 심혈관 질환의 위험을 줄일 수 있습니다.

6. 스트레스 관리: 스트레스는 건강에 부정적인 영향을 미칩니다. 스트레스를 관리하기 위해서는 명상, 운동, 취미활동 등을 통해 스트레스를 해소하는 것이 중요합니다.

7. 스트레스 관리: 스트레스는 건강에 부정적인 영향을 미치는


In [37]:
sent = "건강한 몸을 만들기 위해서는"
generated_text = pipe(sent, max_new_tokens=256, eos_token_id=2, pad_token_id=2)
print(generated_text[0]['generated_text'])

건강한 몸을 만들기 위해서는 적절한 운동과 영양 섭취가 필요합니다.

2. 스트레스 관리: 스트레스는 건강에 부정적인 영향을 미치는 요인 중 하나입니다. 스트레스를 관리하기 위해서는 명상, 운동, 취미활동 등을 통해 스트레스를 해소하는 것이 중요합니다.

3. 수면: 수면 부족은 건강에 부정적인 영향을 미칩니다. 충분한 수면을 취하지 않으면 면역체계가 약화되고, 심혈관 질환의 위험이 증가할 수 있습니다.

4. 영양 섭취: 건강한 식단은 건강한 체중을 유지하는 데 도움이 됩니다. 균형 잡힌 식단은 영양소를 충분히 공급하고, 혈당 조절을 도와줍니다.

5. 운동: 운동은 건강한 체중을 유지하는 데 도움이 됩니다. 유산소 운동과 근력 운동을 병행하면 근육량을 증가시키고, 심혈관 질환의 위험을 줄일 수 있습니다.

6. 스트레스 관리: 스트레스는 건강에 부정적인 영향을 미칩니다. 스트레스를 관리하기 위해서는 명상, 운동, 취미활동 등을 통해 스트레스를 해소하는 것이 중요합니다.

7. 스트레스 관리: 스트레스는 건강에 부정적인 영향을 미치는


In [38]:
chat_sent = '건강한 몸을 만들기 위해서는'
generated_text = pipe(f"chat_sent", max_new_tokens=256, eos_token_id=2, pad_token_id=2) ### ... 이 부분을 완성하시오
print(generated_text[0]['generated_text'])

chat_sent_message ()
         # 메시지를 보내고 응답을 받습니다.
         send_message (메시지)
         # 응답을 확인하고 메시지를 다시 보내십시오.
         send_message (메시지)
         # 응답을 확인하고 메시지를 다시 보내십시오.
         send_message (메시지)
         # 응답을 확인하고 메시지를 다시 보내십시오.
         send_message (메시지)
         # 응답을 확인하고 메시지를 다시 보내십시오.
         send_message (메시지)
         # 응답을 확인하고 메시지를 다시 보내십시오.
         send_message (메시지)
         # 응답을 확인하고 메시지를 다시 보내십시오.
  


In [39]:
chat_sent = '건강한 몸을 만들기 위해서는 어떻게 해야 돼?'
generated_text = pipe(f"### 사용자: {chat_sent}\n\n### 챗봇:", max_new_tokens=256, eos_token_id=2, pad_token_id=2)
print(generated_text[0]['generated_text'])

### 사용자: 건강한 몸을 만들기 위해서는 어떻게 해야 돼?

### 챗봇: 건강한 몸을 만들기 위해서는 다음과 같은 방법을 시도해 볼 수 있습니다.

1. 규칙적인 운동: 규칙적인 운동은 건강한 몸을 유지하는 데 도움이 됩니다. 하루에 30분 이상의 유산소 운동을 규칙적으로 하는 것이 좋습니다.

2. 충분한 수면: 충분한 수면을 취하는 것은 건강한 몸을 유지하는 데 도움이 됩니다. 수면 부족은 스트레스와 피로를 유발할 수 있으므로 충분한 수면을 취하는 것이 중요합니다.

3. 균형 잡힌 식단: 건강한 식단은 건강한 몸을 유지하는 데 도움이 됩니다. 균형 잡힌 식단은 영양소를 충분히 섭취할 수 있도록 도와주며, 영양소의 불균형을 예방할 수 있습니다.

4. 충분한 휴식: 충분한 휴식은 건강한 몸을 유지하는 데 도움이 됩니다. 충분한 휴식은 스트레스를 줄이고, 면역력을 향상시키는 데 도움이 됩니다.

5. 스트레스 관리: 스트레스는 건강에 해로운 영향을 미칠 수 있으므로, 스트레스를 관리하는 것이 중요합니다. 스트레스를 관리하는 방법으로는 명상, 운동, 취미활동 등


In [40]:
chat_sent = '건강한 몸을 만들기 위해서는 어떻게 해야 돼?'
prompt = '당신은 AI 챗봇입니다. 사용자에게 도움이 되고 유익한 내용을 제공해야 합니다. 답변은 길고 자세하며 친절한 설명을 덧붙여서 작성하세요.'
generated_text = pipe(f"{prompt}\n\n### 사용자: {chat_sent}\n\n### 챗봇:", max_new_tokens=256, eos_token_id=2, pad_token_id=2)
print(generated_text[0]['generated_text'])

당신은 AI 챗봇입니다. 사용자에게 도움이 되고 유익한 내용을 제공해야 합니다. 답변은 길고 자세하며 친절한 설명을 덧붙여서 작성하세요.

### 사용자: 건강한 몸을 만들기 위해서는 어떻게 해야 돼?

### 챗봇: 건강한 몸을 만들기 위해서는 다음과 같은 방법을 고려해 볼 수 있습니다.

1. 규칙적인 운동: 규칙적인 운동은 신체의 건강을 유지하는 데 도움이 됩니다. 하루에 30분 이상의 유산소 운동을 포함하여, 근력 운동과 유산소 운동을 함께 진행하는 것이 좋습니다.

2. 충분한 수면: 충분한 수면을 취하는 것은 건강한 몸을 유지하는 데 도움이 됩니다. 수면 부족은 신체의 면역 기능을 저하시키고, 질병에 대한 저항력을 약화시킬 수 있습니다.

3. 균형 잡힌 식단: 균형 잡힌 식단은 건강한 몸을 유지하는 데 도움이 됩니다. 과일, 채소, 단백질, 지방 등 다양한 영양소를 균형 있게 섭취하는 것이 중요합니다.

4. 스트레스 관리: 스트레스는 신체의 건강을 유지하는 데 방해가 될 수 있습니다. 스트레스를 관리하는 방법으로는 명상, 운동, 취미 활동 등이 있습니다.

5. 충분한 휴식: 충분한 휴식은 신체의 건강을 유지하는 데 도움이 됩니다. 충분한 수면, 운동, 휴식 등을 통해 스트레스를 관리하고


In [41]:
print(generated_text[0]['generated_text'].split('### 챗봇:')[1])

 건강한 몸을 만들기 위해서는 다음과 같은 방법을 고려해 볼 수 있습니다.

1. 규칙적인 운동: 규칙적인 운동은 신체의 건강을 유지하는 데 도움이 됩니다. 하루에 30분 이상의 유산소 운동을 포함하여, 근력 운동과 유산소 운동을 함께 진행하는 것이 좋습니다.

2. 충분한 수면: 충분한 수면을 취하는 것은 건강한 몸을 유지하는 데 도움이 됩니다. 수면 부족은 신체의 면역 기능을 저하시키고, 질병에 대한 저항력을 약화시킬 수 있습니다.

3. 균형 잡힌 식단: 균형 잡힌 식단은 건강한 몸을 유지하는 데 도움이 됩니다. 과일, 채소, 단백질, 지방 등 다양한 영양소를 균형 있게 섭취하는 것이 중요합니다.

4. 스트레스 관리: 스트레스는 신체의 건강을 유지하는 데 방해가 될 수 있습니다. 스트레스를 관리하는 방법으로는 명상, 운동, 취미 활동 등이 있습니다.

5. 충분한 휴식: 충분한 휴식은 신체의 건강을 유지하는 데 도움이 됩니다. 충분한 수면, 운동, 휴식 등을 통해 스트레스를 관리하고


## [[실습 14-1]] 자연어 생성과 챗봇을 위한 GPT 모델 비교하기

자연어 생성을 목적으로 하는 GPT 모델은 다음과 같다.
```
pipe = pipeline("text-generation", model="skt/kogpt2-base-v2", device=0)
```

챗봇 용도의 GPT 모델은 다음과 같다.
```
pipe = pipeline("text-generation", model="heegyu/polyglot-ko-1.3b-chat", device=0)
```
<br>

동일한 문장을 각 pipe의 입력으로 사용해보고, 두 결과를 비교해 보시오.



[테스트 입력 문장1]
- (자연어 생성에 적합할 것으로 기대되는) 입력 문장1: "인천국제공항에서 바르셀로나 엘프라트 공항까지 가기 위해서는"

- 자연어 생성 출력 결과:

인천국제공항에서 바르셀로나 엘프라트 공항까지 가기 위해서는 반드시 출국장 인근 주차장까지 가야하고 비행기로 이동할 때에도 탑승한 후 다시 공항에서 내리는 일이 많았다"며 아쉬움을 나타냈다.
특히 “항공사들의 여객들이 공항에서 많이 내려올수록 출국장 주변 도로에서는 차량이 늘어나게 되는데, 이 때문에 공항에서는 차내에서 발생하는 불법 음주운전으로 인한 사고로 인한 사고가 빈발하고 있다”며 “공항에서도 이러한 안전사고가 되풀이되지 않도록 항공사들도 공항과 주차장 같은 교통안전시설을 재정비할 필요가 있다”고 지적했다. 국내 첫 우주왕복선이 발사 하루 만에 상공을 벗어났습니다.
그동안 우리나라는 미확인 물체가 우주선 주변을 스쳐 지나가고 있는 것에 대한 우려가 적지 않았던데요.
나로호가 오는 대로 본격적인 활동에 들어갈 수 있을지를 두고 찬반 논란이 예상됩니다.
김경아 기자가 취재했습니다.

- 챗봇 출력 결과:

인천국제공항에서 바르셀로나 엘프라트 공항까지 가는 방법은 다음과 같습니다.

1. 인천국제공항에서 출발하기:
   - 공항 도착 후, 터미널 1에서 체크인을 합니다.
   - 체크인 후, 터미널 2에서 수하물을 부칩니다.
   - 수하물을 부칠 때, 수하물의 무게를 확인합니다.
   - 수하물을 부칠 때, 수하물의 크기와 무게를 확인합니다.
   - 수하물을 부칠 때, 수하물의 무게와 크기를 확인합니다.
   - 수하물을 부칠 때, 수하물의 무게와 크기를 확인합니다.
   - 수하물을 부칠 때, 수하물의 무게와 크기를 확인합니다.
   - 수하물을 부칠 때, 수하물의 무게와 크기를 확인합니다.
   - 수하물을 부칠 때, 수하물의 무게와 크기를 확인합니다.
   - 수하물을 부칠 때, 수하물의 무게와 크기를 확인합니다.
   - 수하물을 부칠 때, 수하물의 무게와 크기를 확인합니다.
   - 수하물

- 결과 비교: 자연어 생성 결과로 보면 해당 문장이 들어간 기사가 출력됨을 볼 수 있다. 문장을 분석하여 원하는 답변을 내는 모습은 아니다. 반면 챗봇 출력 결과를 보면 사용자의 문장에서 답변으로 원하는 목적은 파악하였으나, 문장이 질문의 형태가 아니기 때문에 답변문장이 길이만큼 반복되어 출력된다.

[테스트 입력 문장2]
- (챗봇 입력으로 적합할 것으로 기대되는) 입력 문장2: "한국에서 스페인에 가려면 어떻게 가야하나요?"

- 자연어 생성 출력 결과:

한국에서 스페인에 가려면 어떻게 가야하나요? 라며 아쉬워하는 아이들을 보며, 많은 아이들이 ‘스쿨 미식가’ ‘저학년 친구집’이라고 답하고 있다. 지난해 7월 '박근혜 정부에서 가장 문제 많은 부처가 복지부‘라는 우스갯소리가 제기됐다면, 올해 정부 내 복지부는 '가장 문제 많은 부처'로 낙인찍혔을 것이란 분석이다.
지난해 9월 박근혜 정부들어 복지부는 '복지예산 대폭 감축' '사회복지 예산 축소' 등 복지분야를 두고 오락가락 기조를 보일 것이라는 전망이 제기됐다.
반면 공무원노조는 '정치적이고 편파적인' 정부 부처라고 지적하며 '공무원 성과관리제도'를 도입해야 한다고 주장하고 있다.
이와 관련해 청와대 '국민행복위원회'는 내년도 공무원들의 복지수준을 올해 수준으로 끌어올리기 위해 내년부터 복지도둑질을 전면 금지하겠다고 밝혔다.
이에 따라 공무원들은 내년부터 복지수당을 월 5만원에서 5만원으로 인상하는 등 성과급 인상을 요구할 것으로 예상된다.
복지부의 내년도 예산안은 올해보다 5억원가량 적은 1조4930억원에 그쳤고, 올해 정부 재정규모(4조9377억원)에 비해서도 1.9%나 삭감된 규모다.
또 내년도 국·공립 보육시설 종사자 처우 개선사업 예산을 올해보다 253

- 챗봇 출력 결과:

 사용자: 한국에서 스페인에 가려면 어떻게 가야하나요?

 챗봇: 한국에서 스페인으로 가려면, 비행기를 타거나 기차를 타야 합니다. 비행기는 대한항공, 아시아나, 타이항공, 에어프랑스, 에어캐나다 등이 있으며, 기차는 KTX, KTX-산천, 새마을호, 무궁화호 등이 있습니다. 비행기는 인천국제공항에서 출발하며, 기차는 청량리역에서 출발합니다.

- 결과 비교: 자연어 생성 결과로 보면 해당 문장이 들어간 기사가 출력됨을 볼 수 있다. 문장을 분석하여 원하는 답변을 내는 모습은 아니다. 반면 챗봇 출력 결과를 보면 사용자가 질문형태로 문장을 사용하였기에 사용자 질문의 목적을 잘 파악하고 답변까지 내는 모습이다.

In [52]:
pipe = pipeline("text-generation", model="skt/kogpt2-base-v2", device=0)

sent1 = "한국에서 스페인에 가려면 어떻게 가야하나요?"
generated_text = pipe(sent1, max_new_tokens=256)
print(generated_text[0]['generated_text'])

한국에서 스페인에 가려면 어떻게 가야하나요? 라며 아쉬워하는 아이들을 보며, 많은 아이들이 ‘스쿨 미식가’ ‘저학년 친구집’이라고 답하고 있다. 지난해 7월 '박근혜 정부에서 가장 문제 많은 부처가 복지부‘라는 우스갯소리가 제기됐다면, 올해 정부 내 복지부는 '가장 문제 많은 부처'로 낙인찍혔을 것이란 분석이다.
지난해 9월 박근혜 정부들어 복지부는 '복지예산 대폭 감축' '사회복지 예산 축소' 등 복지분야를 두고 오락가락 기조를 보일 것이라는 전망이 제기됐다.
반면 공무원노조는 '정치적이고 편파적인' 정부 부처라고 지적하며 '공무원 성과관리제도'를 도입해야 한다고 주장하고 있다.
이와 관련해 청와대 '국민행복위원회'는 내년도 공무원들의 복지수준을 올해 수준으로 끌어올리기 위해 내년부터 복지도둑질을 전면 금지하겠다고 밝혔다.
이에 따라 공무원들은 내년부터 복지수당을 월 5만원에서 5만원으로 인상하는 등 성과급 인상을 요구할 것으로 예상된다.
복지부의 내년도 예산안은 올해보다 5억원가량 적은 1조4930억원에 그쳤고, 올해 정부 재정규모(4조9377억원)에 비해서도 1.9%나 삭감된 규모다.
또 내년도 국·공립 보육시설 종사자 처우 개선사업 예산을 올해보다 253


In [50]:
pipe = pipeline("text-generation", model="heegyu/polyglot-ko-1.3b-chat", device=0)

sent2 = "인천국제공항에서 바르셀로나 엘프라트 공항까지 가기 위해서는 어떻게 해야 돼?"
generated_text = pipe(f"### 사용자: {sent2}\n\n### 챗봇:", max_new_tokens=256, eos_token_id=2, pad_token_id=2)
print(generated_text[0]['generated_text'])

### 사용자: 인천국제공항에서 바르셀로나 엘프라트 공항까지 가기 위해서는 어떻게 해야 돼?

### 챗봇: 인천국제공항에서 바르셀로나 엘프라트 공항까지 가는 방법은 다양합니다. 일반적으로는 공항철도를 이용하는 것이 가장 편리합니다. 공항철도는 인천국제공항역에서 출발하여 인천국제공항역에서 환승하여 공항철도를 이용할 수 있습니다. 공항철도는 일반열차와 급행열차로 구분되며, 일반열차는 일반적으로 인천국제공항역에서 출발하여 인천국제공항역에서 환승하여 공항철도를 이용할 수 있습니다. 또한, 급행열차는 일반열차와 달리 인천국제공항역에서 출발하여 인천국제공항역에서 환승하여 공항철도를 이용할 수 있습니다.


---